<a href="https://colab.research.google.com/github/deepalikonety/Smart-Spoon-Project/blob/main/food_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow keras numpy matplotlib pandas


In [24]:
from google.colab import files
uploaded = files.upload()


Saving dataset.zip to dataset (1).zip


In [25]:
import zipfile
import os

with zipfile.ZipFile('dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')

# Check extracted folders
os.listdir('dataset')


['dataset']

In [56]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_dir = '/content/dataset/dataset'  # Adjust if loading from Google Drive
img_height, img_width = 150, 150
batch_size = 32

datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)

train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 160 images belonging to 5 classes.
Found 40 images belonging to 5 classes.


In [ ]:
# Computer Vision: Food Recognition + Dietary Recommendations

import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# --- Configuration ---
IMG_SIZE = (150, 150)
BATCH_SIZE = 32
NUM_CLASSES = 5

# --- Label Dictionary ---
dietary_info = {
    "biryani": "High sodium from spice mixes. Recommend simulated salt taste.",
    "butter_chicken": "High sodium and fat. Simulated salt advised.",
    "gulab_jamun": "Low sodium, high sugar. No salt simulation needed.",
    "palak_paneer": "Moderate sodium. Mild stimulation suggested.",
    "poha": "Low sodium. Ideal for salt simulation."
}

# --- Image Augmentation ---
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    shear_range=0.2,
    validation_split=0.2
)

train_data = datagen.flow_from_directory(
    "/content/dataset/dataset",  # Replace with your actual path
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    "/content/dataset/dataset",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# --- Model Architecture (Transfer Learning) ---
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# --- Training ---
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    callbacks=[early_stop]
)

# --- Save Model ---
model.save("food_classifier.keras")

# --- Inference Function ---
from tensorflow.keras.preprocessing import image
from PIL import Image

def predict_and_recommend(img_path):
    img = Image.open(img_path).resize(IMG_SIZE)
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction)
    class_label = list(train_data.class_indices.keys())[class_idx]
    print(f"Prediction: {class_label}")
    print(f"Dietary Suggestion: {dietary_info[class_label]}")


In [ ]:
!pip install streamlit pyngrok


In [59]:
!ngrok authtoken 2wgrBct9sO7I7MKvdTXFKtdYYmf_4PcX4E4iT3P1bF3aW4YjP

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [62]:
from pyngrok import ngrok
import os

# Save Streamlit app code to a file
app_code = '''
import streamlit as st
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model

# --- Styling ---
st.set_page_config(page_title="Smart Spoon", layout="centered")
st.markdown(
    """
    <style>
    body {
        background-color: #f0f2f6;
    }
    .main {
        background-color: #ffffff;
        padding: 2rem;
        border-radius: 12px;
        box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.1);
    }
    .stButton>button {
        background-color: #ff7f50;
        color: white;
        font-weight: bold;
        border-radius: 8px;
    }
    </style>
    """,
    unsafe_allow_html=True
)

# Load model and setup
model = load_model("food_classifier.h5", compile=False)
class_labels = ["biryani", "butter_chicken", "gulab_jamun", "palak_paneer", "poha"]
dietary_info = {
    "biryani": "High sodium from spice mixes. Recommend simulated salt taste.",
    "butter_chicken": "High sodium and fat. Simulated salt advised.",
    "gulab_jamun": "Low sodium, high sugar. No salt simulation needed.",
    "palak_paneer": "Moderate sodium. Mild stimulation suggested.",
    "poha": "Low sodium. Ideal for salt simulation."
}

def predict(img):
    img = img.resize((150, 150))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction)
    class_label = class_labels[class_idx]
    return class_label, dietary_info[class_label]

# --- UI ---
st.markdown("<div class='main'>", unsafe_allow_html=True)
st.title("🍽️ Smart Spoon")
st.write("AI-powered food recognition with personalized dietary advice.")

uploaded_file = st.file_uploader("📸 Upload a food image", type=["jpg", "png", "jpeg"])
if uploaded_file:
    img = Image.open(uploaded_file)
    st.image(img, caption="Uploaded Image", use_column_width=True)
    label, advice = predict(img)
    st.success(f"🍛 Prediction: `{label}`")
    st.info(f"🧂 Dietary Advice: {advice}")

    if "simulate salt" in advice.lower():
        if st.button("🔬 Trigger Salt Simulation"):
            st.warning("⚙️ Salt simulation initiated for testing sensory feedback.")

st.markdown("</div>", unsafe_allow_html=True)
'''

# Write to file
with open("app.py", "w") as f:
    f.write(app_code)

# Launch Streamlit and expose via ngrok
!streamlit run app.py &>/content/logs.txt &

import time
time.sleep(5)  # Give Streamlit time to boot up

public_url = ngrok.connect(8501)
print("👇 Open this link to view your app:")
print(public_url)


👇 Open this link to view your app:
NgrokTunnel: "https://18b0-34-68-213-184.ngrok-free.app" -> "http://localhost:8501"
